In [1]:
## IT'S DANGEROUS TO GO ALONE! TAKE THIS:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#comebacktothis wherever I could improve or left something ugly
#gethelp wherever I need it

*In order to give my computer's RAM a break, I broke the wrangling/cleaning up into three notebooks.
This is part one of the miniseries - getting the data from the United States' National Oceanic and Atmospheric Administration reformatted, cleaned, and into one single dataframe for the 20 years of data we will be working on.*

*On that note, the 'rough' csv's ('roughMinTemp.csv', etc) have been zipped and transferred to the coding storage folder on my D drive, in the Capstone subfolder:*
* D:\gdsak\Coding_Storage\Capstone\roughWeathers*

*And the cleaned csv's ('avg_temp.csv', etc) are also zipped together in the same location:*
* D:\gdsak\Coding_Storage\Capstone\cleanWeathers

# NOAA weather data

## Data Sources

We have three data sources, each with slightly different types of data. The goal is to have either weekly (with imputed/interpolated weather data) or monthly (with binned and averaged drought data)

- **NOAA Weather data** - We'll focus on this
    * This is 4 separate text files. I need to slice out the number that signifies what the data *is* from the fips/year column for each dataframe. Then pivot the rest of the columns into two vertical columns for month and precip/avg temp/min temp/max temp instead of the 12 columns (one per month) format I have now. Then I should be able to combine the 4 dataframes using the fips/year column as an index, ad drop the extra month columns

- US Drought monitor

- USDA/NASS Census and Survey of Ag data

---

In [2]:
min_temp = pd.read_csv('data/roughMinTemp.csv')
min_temp

FileNotFoundError: [Errno 2] No such file or directory: 'data/roughMinTemp.csv'

# Test Drive #1:
### Getting the Weather data into tidy columns.

My weather data started out as a text file that I converted to .csv in the acquisition notebook. I started with 13 columns:
* The first column was for the state/county FIPS # combined with a two digit number indicating the type of data and the year in YYYY format.

* The next 12 columns were for each month of the year.

What I want is a 'tidy' dataframe. The notion of tidy data came from an introductory book on **R***. The goal of tidying up one's data is to have your data following these three rules:

1. Each variable must have its own column
2. Each observation must have its own row
3. Each value must have its own cell

This is accomplished with a handful of functions from the tidyverse library. For my purposes the **gather** and **spread** functions are most salient. 

**Gather** will take a number of rows that are holding the same variable and pivot them vertically. In my case, gathering all of the months and creating a single 'month' column and a single 'min_temp' column (or max_temp/precip, etc). Knowing that this function is commonplace in R led me to search for examples of code that accomplishes this same task in pandas. As has thusfar been the case, if I thought of something I'd like to code, someone has *definitely* already written that code.

*R for Data Science by Hadley Wickham & Garrett Grolemund

In [ ]:
#test driving a gather function a la from someone else:
# https://gist.github.com/derekpowell/5f97dabdd0730e68380fa1a00cd34ac4

def gather( df, key, value, cols ):
    id_vars = [ col for col in df.columns if col not in cols ]
    id_values = cols
    var_name = key
    value_name = value
    return pd.melt( df, id_vars, id_values, var_name, value_name )

In [ ]:
#making a 5 row test set for translations and column splitting
min_temp_small = min_temp[:5].copy()
min_temp_small

In [ ]:
#
gather(df=min_temp_small,
       key='Month',
       value='mintemp',
       cols=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
#yep, that works, now lets fix the first column

Et voila, the borrowed 'gather' function performed as desired. Now to move on to fixing the first column.

The 'FIPSXXYear column for all four weather csv's need to be split up into three columns:

* FIPS - this is currently either 4 or 5 digits, but *should* be 5 digits for all values. The first two digits are for the state and the last 3 digits for the county in that state. I will need to add a leading zero whenever the FIPS number is only 4 digits long.
* 28/27/02/01 - the 2 digit code that signifies what data is held in dataframe. This is a holdover from downloading the data from the NOAA database, I'll be dropping this column.
* Year - the year in YYYY format, this column will be kept as is and once I have a tidy dataframe with all four of my weather variables I can slice out a copy with data from 2002 to 2021.

In [ ]:
#setting the fips/year column to be a string
min_temp_small['FIPS28Year'] = min_temp_small['FIPS28Year'].astype(str)

In [ ]:
#making sure it worked
min_temp_small.info()

In [ ]:
#adding in the leading zero
min_temp_small['FIPS28Year'] = min_temp_small['FIPS28Year'].str.zfill(11)
min_temp_small

In [ ]:
#after some googling for ways to break up a string with regex (because using '28' as a delimiter would've been bad/wrong)
#I found an answer on stackexchange: 
#https://stackoverflow.com/questions/25252200/how-to-split-a-column-based-on-several-string-indices-using-pandas
#and tested it out in regex101 until I got the below code to behave

min_temp_small['FIPS28Year'].str.extract('(.{5})(.{2})(.{4})')

In [ ]:
#creating new columns with the extracted data
min_temp_small[['FIPS', '28', 'Year']] = min_temp_small['FIPS28Year'].str.extract('(.{5})(.{2})(.{4})')

In [ ]:
#checking my work
min_temp_small

In [ ]:
#adding in the gather function
min_temp_small = gather(df=min_temp_small,
       key='Month',
       value='mintemp',
       cols=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
min_temp_small

In [ ]:
#and dropping the FIPS28Year and 28 columns
min_temp_small = min_temp_small.drop(columns=['FIPS28Year', '28'])
min_temp_small

The last thing I will need is to create an index column that merges the FIPS, year, and month so it is easy to join the dataframes together into one weather dataframe

In [ ]:
#recombining the FIPS and year columns so I have something to join the columns on
min_temp_small['FIPSYearMonth'] = min_temp_small['FIPS'] + min_temp_small['Year'] + min_temp_small['Month']
min_temp_small

Now, to load in the other three sets of weather data.

In [ ]:
max_temp = pd.read_csv('data/roughMaxTemp.csv')
max_temp

In [ ]:
avg_temp = pd.read_csv('data/roughAvgTemp.csv')
avg_temp

In [ ]:
precip = pd.read_csv('data/roughPrecip.csv')
precip

### Cleaning note:
The Temperature data has missing values represented with **-99.99** (an impossibly low temperature in fahrenheit for the contiguous USA), while the precipitation data has missing values represented with **-9.99** (a distinctly possible temperature, one that is unfortunately common in the author's hometown in January and February).

Because of this difference in how missing values are presented, we will pre-clean the precipitation data. Because we are mainly interested in the years after 2000, we can wait to do this cleaning until the end of the reformatting of the precip dataframe.

Let's start with precip! We have 6 steps to clean up each of the weather dataframes: 

# COMEBACKTOTHIS 
maybe turn this heap into a function, eh?

**1. Convert to string**
    - min_temp_small['FIPS28Year'] = min_temp_small['FIPS28Year'].astype(str)

**2. Add leading zeros**
    - min_temp_small['FIPS28Year'] = min_temp_small['FIPS28Year'].str.zfill(11)

**3. Gather the month columns**
    - min_temp_small = gather(df=min_temp_small,
       key='Month',
       value='mintemp',
       cols=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])

**4. Extract and create new columns**
    - min_temp_small[['FIPS', '28', 'Year']] = min_temp_small['FIPS28Year'].str.extract('(.{5})(.{2})(.{4})')


**5. Drop the old combined column and the unnecessary label column**
    - min_temp_small = min_temp_small.drop(columns=['FIPS28Year', '28'])

**6. Make an column from the FIPS, Year, and Month so when we join everything we have a key to join on**
    - min_temp_small['FIPSYearMonth'] = min_temp_small['FIPS'] + min_temp_small['Year'] + min_temp_small['Month']



### Reformatting precip

In [ ]:
#Convert to string and add leading zeros
precip['FIPS01Year'] = precip['FIPS01Year'].astype(str)
precip['FIPS01Year'] = precip['FIPS01Year'].str.zfill(11)
precip

In [ ]:
#gather the month data into one column
precip = gather(df=precip,
       key='Month',
       value='precip',
       cols=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])

In [ ]:
#extracting and adding newly split-up columns
precip[['FIPS', '01', 'Year']] = precip['FIPS01Year'].str.extract('(.{5})(.{2})(.{4})')
precip

In [ ]:
#dropping the extra columns, adding in the joining column
precip = precip.drop(columns=['FIPS01Year', '01'])
precip['FIPSYearMonth'] = precip['FIPS'] + precip['Year'] + precip['Month']
precip

### Pre-wash cycle
Our precip dataframe is in the right format, now lets check for missing data. If there is quite a bit of it outside the 20 years we are planning on using, we can cut out the chunk of data we are going to use and perform the changes there.

In [ ]:
#check for missing values
precip[precip['precip']==-9.99]

In [ ]:
#check for missing values that aren't in 2022
precip[
    (precip['precip']==-9.99) & (precip['Year']!='2022')
]

In [ ]:
#checking for nulls
precip.isnull().sum()

It seems that the precipitation data is whole, apart from the last five months in 2022-- which makes sense, considering the data was pulled in August of 2022. We can check the unique number of FIPS, it should be ~3,000, and multiply that by 5. If it's close to the total we are seeing for the first search, then we should have complete data.

In [ ]:
print(f"There are {(precip['FIPS'].nunique())} unique FIPS numbers.\n\
There theoretically are 5 months missing for each, so we should have {(precip['FIPS'].nunique())*5} rows with -9.99.\n\
That matches the row count above of {precip[precip['precip']==-9.99].shape[0]}.")

In [ ]:
precip.info()

Wonderful! The precip dataframe is clean and complete. We can update the data types for Month and Year to integers and create a copy with all of the years from 2002 through 2021. We'll call this precip20. 

Then we can repeat the reformatting process for the other three dataframes and combine them.

In [ ]:
precip['Month'] = precip['Month'].astype(int)
precip['Year'] = precip['Year'].astype(int)
precip.info()

In [ ]:
precip20 = precip[(precip['Year']>2001) & (precip['Year']!=2022)].copy()
precip20

### Reformatting max_temp

In [ ]:
#Checking the two digit number that splits the FIPS and year values for max_temp
max_temp

In [ ]:
#Convert to string and add leading zeros
max_temp['FIPS27Year'] = max_temp['FIPS27Year'].astype(str)
max_temp['FIPS27Year'] = max_temp['FIPS27Year'].str.zfill(11)
max_temp

In [ ]:
#gather the month data into one column
max_temp = gather(df=max_temp,
       key='Month',
       value='maxtemp',
       cols=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
max_temp

In [ ]:
#extracting and adding newly split-up columns
max_temp[['FIPS', '27', 'Year']] = max_temp['FIPS27Year'].str.extract('(.{5})(.{2})(.{4})')
max_temp

In [ ]:
#dropping the extra columns, adding in the joining column
max_temp = max_temp.drop(columns=['FIPS27Year', '27'])
max_temp['FIPSYearMonth'] = max_temp['FIPS'] + max_temp['Year'] + max_temp['Month']
max_temp

In [ ]:
max_temp['Month'] = max_temp['Month'].astype(int)
max_temp['Year'] = max_temp['Year'].astype(int)
max_temp.info()

In [ ]:
#setting up a subsegment for 2002 through 2021
max_temp20 = max_temp[(max_temp['Year']>2001) & (max_temp['Year']!=2022)].copy()
max_temp20

In [ ]:
#checking for missing values in max_temp20
max_temp20[max_temp20['maxtemp']==-99.9]

In [ ]:
#checking for nulls
max_temp20.isnull().sum()

### Reformatting min_temp

In [ ]:
#Checking the two digit number that splits the FIPS and year values for max_temp
min_temp

In [ ]:
#Convert to string and add leading zeros
min_temp['FIPS28Year'] = min_temp['FIPS28Year'].astype(str)
min_temp['FIPS28Year'] = min_temp['FIPS28Year'].str.zfill(11)
min_temp

In [ ]:
#gather the month data into one column
min_temp = gather(df=min_temp,
       key='Month',
       value='mintemp',
       cols=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
min_temp

In [ ]:
#extracting and adding newly split-up columns
min_temp[['FIPS', '28', 'Year']] = min_temp['FIPS28Year'].str.extract('(.{5})(.{2})(.{4})')
min_temp

In [ ]:
#dropping the extra columns, adding in the joining column
min_temp = min_temp.drop(columns=['FIPS28Year', '28'])
min_temp['FIPSYearMonth'] = min_temp['FIPS'] + min_temp['Year'] + min_temp['Month']
min_temp

In [ ]:
min_temp['Month'] = min_temp['Month'].astype(int)
min_temp['Year'] = min_temp['Year'].astype(int)
min_temp.info()

In [ ]:
#setting up a subsegment for 2002 through 2021
min_temp20 = min_temp[(min_temp['Year']>2001) & (min_temp['Year']!=2022)].copy()
min_temp20

In [ ]:
#checking for missing values in min_temp20
min_temp20[min_temp20['mintemp']==-99.9]

In [ ]:
#checking for nulls
min_temp20.isnull().sum()

### Reformatting avg_temp
##### last but not least

In [ ]:
#Checking the two digit number that splits the FIPS and year values for max_temp
avg_temp

In [ ]:
#Convert to string and add leading zeros
avg_temp['FIPS02Year'] = avg_temp['FIPS02Year'].astype(str)
avg_temp['FIPS02Year'] = avg_temp['FIPS02Year'].str.zfill(11)
avg_temp

In [ ]:
#gather the month data into one column
avg_temp = gather(df=avg_temp,
       key='Month',
       value='avgtemp',
       cols=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'])
avg_temp

In [ ]:
#extracting and adding newly split-up columns
avg_temp[['FIPS', '02', 'Year']] = avg_temp['FIPS02Year'].str.extract('(.{5})(.{2})(.{4})')
avg_temp

In [ ]:
#dropping the extra columns, adding in the joining column
avg_temp = avg_temp.drop(columns=['FIPS02Year', '02'])
avg_temp['FIPSYearMonth'] = avg_temp['FIPS'] + avg_temp['Year'] + avg_temp['Month']
avg_temp

In [ ]:
avg_temp['Month'] = avg_temp['Month'].astype(int)
avg_temp['Year'] = avg_temp['Year'].astype(int)
avg_temp.info()

In [ ]:
#setting up a subsegment for 2002 through 2021
avg_temp20 = avg_temp[(avg_temp['Year']>2001) & (avg_temp['Year']!=2022)].copy()
avg_temp20

In [ ]:
#checking for missing values in avg_temp20
avg_temp20[avg_temp20['avgtemp']==-99.9]

In [ ]:
#checking for nulls
avg_temp20.isnull().sum()

I now have four, tidied and abridged dataframes that can be combined into one dataframe, **weather**.
I have a column set up in each dataframe that can be used as a key for merging them, so I can actually drop the Month, FIPS, and Year columns from most of the dataframes. Luckily, I don't have missing data for any of the dataframes I will use for our analysis.

I'm going to keep precip20 whole and drop the columns from the temperature dataframes.

In [ ]:
#dropping the unnecessary columns
avg_temp20 = avg_temp20.drop(columns=['Month', 'Year', 'FIPS'])
min_temp20 = min_temp20.drop(columns=['Month', 'Year', 'FIPS'])
max_temp20 = max_temp20.drop(columns=['Month', 'Year', 'FIPS'])

In [ ]:
#combining precip and avg_temp dataframes on the 'FIPSYearMonth' columns
weather = pd.merge(precip20, avg_temp20, on='FIPSYearMonth')
weather

In [ ]:
#Adding the min temp dataframe
weather = pd.merge(weather, min_temp20, on='FIPSYearMonth')
weather

In [ ]:
#adding the max temp dataframe
weather = pd.merge(weather, max_temp20, on='FIPSYearMonth')
weather

In [ ]:
#rearranging the columns
weather = weather[['FIPSYearMonth', 'FIPS', 'Year', 'Month', 'precip', 'mintemp', 'maxtemp', 'avgtemp']]
weather

In [ ]:
weather.info()

In [ ]:
#double checking in case we creating null values
weather.isnull().sum()
#bless the NOAA and their impeccably complete data

All of the weather data is clean and the 20 year period we will be focusing on is assembled into the dataframe, **weather**. Now to export the four whole dataframes and the **weather** dataframe to fresh csv files. To save drive space, we can now zip the old data, move it to a storage drive and leave ourselves a note up at the top of the notebook so we can find it easily. Same goes for the cleaned data that spans back to 1895-- we likely wont use it in this iteration of the project but we can put it on ice (...compress it) in case we need to use it later.

We can remake the FIPSYearMonth column for each of these if need be, so prior to exporting we can save a bit of space and drop that column.

In [ ]:
#dropping the FIPSYearMonth column from
#precip = precip.drop(columns=['FIPSYearMonth'])
#avg_temp = avg_temp.drop(columns=['FIPSYearMonth'])
#min_temp = min_temp.drop(columns=['FIPSYearMonth'])
#max_temp = max_temp.drop(columns=['FIPSYearMonth'])

In [ ]:
precip.info()

In [ ]:
#spit out the full cleaned dataframes and the combined weather dataframe
#avg_temp.to_csv('avg_temp.csv', index=False)
#min_temp.to_csv('min_temp.csv', index=False)
#max_temp.to_csv('max_temp.csv', index=False)
#precip.to_csv('precip.csv', index=False)
#weather.to_csv('weather.csv', index=False)